# 🚀 StairVision YOLO Training Pipeline

**Project**: CS663 Mobile Vision - Stair Detection Android App

**Goal**: Train YOLOv8 model on stairs dataset and export to TensorFlow Lite for Android

**Requirements**:
- Dataset uploaded to Google Drive: `MyDrive/stairs_yolo_dataset.zip`
- GPU Runtime: T4 GPU (Runtime → Change runtime type → T4 GPU)

---

## 📋 Training Steps
1. Install dependencies
2. Mount Google Drive
3. Extract and verify dataset
4. Train YOLOv8 model (100 epochs)
5. Evaluate performance
6. Export to TFLite Float32
7. Save model to Google Drive

**Estimated Time**: 1-2 hours

---
## CELL 1: Install Dependencies and Mount Drive

In [ ]:
# Install required packages (with PyTorch 2.6 compatibility fix)
!pip install torch==2.1.0 torchvision==0.16.0 -q
!pip install ultralytics==8.1.0 -q
!pip install tensorflow==2.15.0 -q

print("✅ Package installation complete")

# Import libraries
from google.colab import drive
from ultralytics import YOLO
import os
import shutil
from IPython.display import Image, display
import torch

# Fix for PyTorch 2.6 weights_only issue
print(f"PyTorch version: {torch.__version__}")

# Mount Google Drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted successfully")
print("✅ Ready for training!")

---
## CELL 2: Extract and Verify Dataset

In [ ]:
# Dataset location in Google Drive
dataset_zip = '/content/drive/MyDrive/stairs_yolo_dataset.zip'

# Check if dataset exists
if not os.path.exists(dataset_zip):
    print("❌ ERROR: Dataset not found!")
    print(f"   Expected location: {dataset_zip}")
    print("\n📋 Upload Instructions:")
    print("   1. Go to drive.google.com")
    print("   2. Upload stairs_yolo_dataset.zip to MyDrive/")
    print("   3. Refresh and run this cell again")
    raise FileNotFoundError("Dataset not found in Google Drive")

print(f"✅ Found dataset: {dataset_zip}")
print(f"   Size: {os.path.getsize(dataset_zip) / 1024 / 1024:.1f} MB")

# Extract dataset
print("\n📦 Extracting dataset...")
!unzip -o -q {dataset_zip} -d /content/

dataset_path = '/content/stairs_yolo_dataset'

# Verify directory structure
print("\n📁 Dataset structure:")
!ls -R {dataset_path} | head -40

# Count files
train_imgs = len([f for f in os.listdir(f'{dataset_path}/images/train') if f.endswith('.jpg')])
val_imgs = len([f for f in os.listdir(f'{dataset_path}/images/val') if f.endswith('.jpg')])
train_labels = len([f for f in os.listdir(f'{dataset_path}/labels/train') if f.endswith('.txt')])
val_labels = len([f for f in os.listdir(f'{dataset_path}/labels/val') if f.endswith('.txt')])

print("\n📊 Dataset Statistics:")
print(f"   Training:   {train_imgs} images | {train_labels} labels")
print(f"   Validation: {val_imgs} images | {val_labels} labels")
print(f"   Total:      {train_imgs + val_imgs} images")

# Verify match
if train_imgs == train_labels and val_imgs == val_labels:
    print("\n✅ Dataset verification PASSED - Ready for training!")
else:
    print("\n⚠️  Warning: Image/label count mismatch")
    print(f"   Train: {train_imgs} images vs {train_labels} labels")
    print(f"   Val: {val_imgs} images vs {val_labels} labels")

# Fix data.yaml paths for Colab
print("\n🔧 Updating data.yaml paths for Colab...")
yaml_path = f'{dataset_path}/data.yaml'
with open(yaml_path, 'w') as f:
    f.write(f'''# Stair Detection Dataset\n''')
    f.write(f'''path: {dataset_path}\n''')
    f.write(f'''train: images/train\n''')
    f.write(f'''val: images/val\n''')
    f.write(f'''\n''')
    f.write(f'''# Number of classes\n''')
    f.write(f'''nc: 1\n''')
    f.write(f'''\n''')
    f.write(f'''# Class names\n''')
    f.write(f'''names:\n''')
    f.write(f'''  0: stairs\n''')
print(f"✅ data.yaml updated with Colab paths")

---
## CELL 3: Initialize YOLOv8 Model

In [ ]:
# Load pretrained YOLOv8 nano model
print("📥 Downloading YOLOv8n pretrained weights...")

# Fix for PyTorch 2.6 weights_only security issue
import torch.serialization
from ultralytics.nn.tasks import DetectionModel

# Add safe globals for YOLO model loading
torch.serialization.add_safe_globals([DetectionModel])
print("🔧 Applied PyTorch 2.6 compatibility fix")

# Now load the model
model = YOLO('yolov8n.pt')

print("\n✅ YOLOv8n model loaded successfully")
print(f"   Model type: YOLOv8 Nano")
print(f"   Parameters: {sum(p.numel() for p in model.model.parameters()):,}")
print(f"   Optimized for: Mobile/Edge devices")
print(f"\n🎯 Ready to train on stairs dataset!")

---
## CELL 4: Train the Model (⏱️ Takes 1-2 hours)

In [ ]:
print("🔥 Starting training...")
print("   This will take approximately 1-2 hours")
print("   You can monitor progress below\n")

# Training configuration
results = model.train(
    data='/content/stairs_yolo_dataset/data.yaml',
    epochs=100,              # Number of training epochs
    imgsz=640,              # Image size (must match Android: 640x640)
    batch=16,               # Batch size (reduce to 8 if GPU memory issues)
    device=0,               # Use GPU 0
    project='stair_detection',
    name='yolov8n_stairs_v1',
    patience=20,            # Early stopping patience
    save=True,              # Save checkpoints
    plots=True,             # Generate training plots
    single_cls=True,        # Single class optimization (stairs only)
    exist_ok=True,
    
    # Data augmentation (helps with small dataset)
    hsv_h=0.015,           # Hue augmentation
    hsv_s=0.7,             # Saturation augmentation
    hsv_v=0.4,             # Value/brightness augmentation
    degrees=10,            # Random rotation (±10 degrees)
    translate=0.1,         # Random translation (10%)
    scale=0.5,             # Random scale (50% variation)
    shear=0,               # Shearing (disabled)
    perspective=0.0,       # Perspective transform (disabled)
    flipud=0.0,            # Vertical flip probability (disabled)
    fliplr=0.5,            # Horizontal flip probability (50%)
    mosaic=1.0,            # Mosaic augmentation (100%)
    mixup=0.1,             # Mixup augmentation (10%)
)

print("\n✅ Training completed!")
print(f"   Results saved to: /content/stair_detection/yolov8n_stairs_v1/")

---
## CELL 5: Evaluate Model Performance

In [ ]:
# Validate on validation set
print("📊 Evaluating model on validation set...\n")
metrics = model.val()

# Display metrics
print("\n" + "="*60)
print("📈 MODEL PERFORMANCE METRICS")
print("="*60)
print(f"\n🎯 Detection Metrics:")
print(f"   mAP@50:     {metrics.box.map50:.4f} ({metrics.box.map50*100:.2f}%)")
print(f"   mAP@50-95:  {metrics.box.map:.4f} ({metrics.box.map*100:.2f}%)")
print(f"   Precision:  {metrics.box.p:.4f} ({metrics.box.p*100:.2f}%)")
print(f"   Recall:     {metrics.box.r:.4f} ({metrics.box.r*100:.2f}%)")

# Performance assessment
print(f"\n💡 Performance Assessment:")
if metrics.box.map50 > 0.7:
    print("   ✅ EXCELLENT performance (>70%)")
    print("   → Model is ready for production deployment")
elif metrics.box.map50 > 0.5:
    print("   ✅ GOOD performance (>50%)")
    print("   → Model is suitable for deployment")
elif metrics.box.map50 > 0.3:
    print("   ⚠️  ACCEPTABLE performance (>30%)")
    print("   → Consider training longer or adding more data")
else:
    print("   ❌ POOR performance (<30%)")
    print("   → Needs more training epochs or more training data")

print("\n" + "="*60)

---
## CELL 6: Display Training Results & Visualizations

In [ ]:
# Results directory
results_dir = '/content/stair_detection/yolov8n_stairs_v1'

# Display training curves
print("📈 Training Curves:")
display(Image(f'{results_dir}/results.png'))

print("\n📊 Confusion Matrix:")
display(Image(f'{results_dir}/confusion_matrix.png'))

print("\n🎯 Validation Batch Predictions:")
display(Image(f'{results_dir}/val_batch0_pred.jpg'))

print("\n✅ All training visualizations displayed")

---
## CELL 7: Test Predictions on Sample Images

In [ ]:
# Test predictions on validation images
print("🧪 Running predictions on validation set...")

test_images = '/content/stairs_yolo_dataset/images/val'
results = model.predict(test_images, save=True, conf=0.3)

print(f"✅ Predictions saved to: /content/runs/detect/predict/")

# Display sample predictions
from glob import glob
pred_images = sorted(glob('/content/runs/detect/predict/*.jpg'))

if pred_images:
    print(f"\n🖼️  Displaying sample predictions ({len(pred_images)} total):\n")
    
    # Show first 3 predictions
    for i, img_path in enumerate(pred_images[:3]):
        print(f"Sample {i+1}:")
        display(Image(img_path, width=600))
        print()
else:
    print("⚠️  No prediction images found")

---
## CELL 8: Export to TensorFlow Lite (Float32 - CRITICAL)

In [ ]:
# Load best trained model
best_model_path = '/content/stair_detection/yolov8n_stairs_v1/weights/best.pt'
print(f"📦 Loading best model from: {best_model_path}")
model = YOLO(best_model_path)

print("\n🔄 Exporting to TensorFlow Lite (Float32)...")
print("   IMPORTANT: Using float32 (NOT int8) for Android compatibility\n")

# Export to TFLite with float32 precision
tflite_path = model.export(
    format='tflite',
    imgsz=640,
    int8=False,  # CRITICAL: Use float32, not int8 quantization
    data='/content/stairs_yolo_dataset/data.yaml'
)

print(f"✅ TFLite model exported successfully!")
print(f"   Path: {tflite_path}")
print(f"   Size: {os.path.getsize(tflite_path) / 1024 / 1024:.2f} MB")

# Rename to Android-expected name
final_model_path = '/content/stair_detection/yolov8n_stairs_v1/weights/stair_yolo_best_float32.tflite'
shutil.copy(tflite_path, final_model_path)

print(f"✅ Model renamed to: stair_yolo_best_float32.tflite")

---
## CELL 9: Verify TFLite Model Format (CRITICAL CHECK)

In [ ]:
import tensorflow as tf

# Load TFLite model
print("🔍 Verifying TFLite model format...\n")
interpreter = tf.lite.Interpreter(model_path=final_model_path)
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("="*60)
print("🔍 TFLite Model Specification")
print("="*60)

print("\n📥 Input Tensor:")
print(f"   Shape:     {input_details[0]['shape']}")
print(f"   Type:      {input_details[0]['dtype']}")
print(f"   Expected:  [1, 640, 640, 3] float32")

print("\n📤 Output Tensor:")
print(f"   Shape:     {output_details[0]['shape']}")
print(f"   Type:      {output_details[0]['dtype']}")
print(f"   Expected:  [1, 5, 8400] float32")
print(f"   Format:    [class, x, y, w, h] × 8400 anchors")

# Validate shapes
expected_input = [1, 640, 640, 3]
expected_output = [1, 5, 8400]

input_shape = list(input_details[0]['shape'])
output_shape = list(output_details[0]['shape'])

input_ok = input_shape == expected_input
output_ok = output_shape == expected_output

print("\n🔍 Validation Results:")
print(f"   Input shape:  {'✅ PASS' if input_ok else '❌ FAIL'}")
print(f"   Output shape: {'✅ PASS' if output_ok else '❌ FAIL'}")

if input_ok and output_ok:
    print("\n✅ MODEL FORMAT VERIFIED - Ready for Android deployment!")
    print("   → This model is compatible with your Android app")
else:
    print("\n❌ MODEL FORMAT MISMATCH - DO NOT use this model!")
    print("   → Re-export with correct settings")
    if not input_ok:
        print(f"   → Input: Got {input_shape}, expected {expected_input}")
    if not output_ok:
        print(f"   → Output: Got {output_shape}, expected {expected_output}")

print("\n" + "="*60)

---
## CELL 10: Save Model and Results to Google Drive

In [ ]:
print("💾 Saving trained model to Google Drive...\n")

# Save TFLite model
drive_model_path = '/content/drive/MyDrive/stair_yolo_best_float32.tflite'
shutil.copy(final_model_path, drive_model_path)

print(f"✅ Model saved to Google Drive!")
print(f"   Location: MyDrive/stair_yolo_best_float32.tflite")
print(f"   Size: {os.path.getsize(drive_model_path) / 1024 / 1024:.2f} MB")

# Save training results
print(f"\n💾 Saving training results...")
drive_results_dir = '/content/drive/MyDrive/stair_training_results'
os.makedirs(drive_results_dir, exist_ok=True)

# Copy result files
results_files = [
    'results.png',
    'confusion_matrix.png',
    'val_batch0_pred.jpg',
    'F1_curve.png',
    'PR_curve.png',
    'P_curve.png',
    'R_curve.png'
]

for filename in results_files:
    src = f'{results_dir}/{filename}'
    if os.path.exists(src):
        shutil.copy(src, f'{drive_results_dir}/{filename}')
        print(f"   ✅ {filename}")

print(f"\n✅ All results saved to: MyDrive/stair_training_results/")
print(f"\n🎉 Training pipeline complete!")

---
## CELL 11: Final Summary & Next Steps

In [ ]:
print("\n" + "="*70)
print("🎉 TRAINING COMPLETE - READY FOR ANDROID DEPLOYMENT")
print("="*70)

print(f"\n📊 Final Model Performance:")
print(f"   mAP@50:     {metrics.box.map50*100:.2f}%")
print(f"   mAP@50-95:  {metrics.box.map*100:.2f}%")
print(f"   Precision:  {metrics.box.p*100:.2f}%")
print(f"   Recall:     {metrics.box.r*100:.2f}%")

print(f"\n📁 Files Created in Google Drive:")
print(f"   1. stair_yolo_best_float32.tflite  ({os.path.getsize(drive_model_path)/1024/1024:.1f} MB)")
print(f"   2. stair_training_results/  (plots and metrics)")

print(f"\n🔄 Next Steps for Android Integration:")
print(f"\n   1️⃣  Download Model from Google Drive:")
print(f"      → Open: drive.google.com")
print(f"      → Find: stair_yolo_best_float32.tflite")
print(f"      → Download to: ~/Downloads/")

print(f"\n   2️⃣  Copy to Android Project:")
print(f"      → Terminal: cp ~/Downloads/stair_yolo_best_float32.tflite \\")
print(f"                     ~/AndroidStudioProjects/StairVision/app/src/main/assets/")

print(f"\n   3️⃣  Disable Demo Mode:")
print(f"      → Edit: app/src/main/java/com/example/stairvision/DetectionConfig.kt")
print(f"      → Change: ENABLE_DEMO_MODE = false")

print(f"\n   4️⃣  Rebuild Android App:")
print(f"      → ./gradlew clean assembleDebug")
print(f"      → adb install -r app/build/outputs/apk/debug/app-debug.apk")

print(f"\n   5️⃣  Test Real Stair Detection:")
print(f"      → Launch app on Android device")
print(f"      → Point camera at REAL stairs")
print(f"      → Verify: \"STAIRS DETECTED ✅\" appears")
print(f"      → Check: Audio navigation works")
print(f"      → Confirm: Colored bounding boxes show")

print(f"\n✅ Success Criteria:")
print(f"   • Detects real stairs (not images)")
print(f"   • Confidence: 40-95%")
print(f"   • Audio speaks stair type and distance")
print(f"   • Bounding boxes colored by urgency")
print(f"   • No false positives on non-stairs")

print("\n" + "="*70)
print("🎓 Ready for CS663 Project Demo!")
print("="*70)